# Backtesting the Strategy

In [74]:
# Importing additional necessary libraries for quant metrics and plotting
from pyfolio.timeseries import perf_stats
import plotly.graph_objs as go
import os
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import zscore
import yfinance as yf
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


In [75]:
def backtest(df_trading, asset1 : str, asset2 : str, beta: float):
    metrics = {}

    df_trading['spread'] = df_trading[asset2] - beta * df_trading[asset1]
    df_trading['zscore'] = zscore(df_trading['spread'])

    UL = df_trading['zscore'].mean() + df_trading['zscore'].std()
    LL = df_trading['zscore'].mean() - df_trading['zscore'].std()

    holding_position = False  # Flag to indicate if we are holding a position
    df_trading['asset1_signal'] = 0
    df_trading['asset2_signal'] = 0

    for i in range(1, len(df_trading)):
        if not holding_position:
            if df_trading['zscore'].iloc[i] > UL:
                df_trading['asset1_signal'].iloc[i] = -beta
                df_trading['asset2_signal'].iloc[i] = 1
                holding_position = True  # Now holding a position

            elif df_trading['zscore'].iloc[i] < LL:
                df_trading['asset1_signal'].iloc[i] = beta
                df_trading['asset2_signal'].iloc[i] = -1
                holding_position = True  # Now holding a position

        elif holding_position:
            if LL <= df_trading['zscore'].iloc[i] <= UL:
                # Closing the trade
                df_trading['asset1_signal'].iloc[i] = -df_trading['asset1_signal'].iloc[i-1]  # Reverse the last trade
                df_trading['asset2_signal'].iloc[i] = -df_trading['asset2_signal'].iloc[i-1]
                holding_position = False  # No longer holding a position

        # Daily returns
    df_trading['asset1_signal'] = df_trading['asset1_signal'].shift(1)
    df_trading['asset2_signal'] = df_trading['asset2_signal'].shift(1)

    df_trading['asset1_returns'] = df_trading[asset1].pct_change() * df_trading['asset1_signal']
    df_trading['asset2_returns'] = df_trading[asset2].pct_change() * df_trading['asset2_signal']

    df_trading['portfolio_returns'] = df_trading['asset1_returns'] + df_trading['asset2_returns']

    # Quantitative metrics
    stats = perf_stats(df_trading['portfolio_returns'].dropna())
    metrics['CAGR'] = stats['Annual return']
    metrics['Sharpe ratio'] = stats['Sharpe ratio']
    metrics['Max Drawdown'] = stats['Max drawdown']
    metrics['Number of Trades'] = df_trading['asset1_signal'].ne(0).sum()  # Counting non-zero entries

    return metrics, df_trading

In [76]:
# Function to save the trading signals graph
def save_plotly_graph(df, asset1, asset2):
    # Create directory if it doesn't exist
    if not os.path.exists('img/signals'):
        os.makedirs('img/signals')

    # Create the figure
    fig = go.Figure()

    # Add z-score trace
    fig.add_trace(go.Scatter(x=df.index, y=df['zscore'], mode='lines', name='Z-Score'))

    # Add upper and lower limits as dashed lines
    UL = df['zscore'].mean() + df['zscore'].std()
    LL = df['zscore'].mean() - df['zscore'].std()

    fig.add_trace(go.Scatter(x=df.index, y=[UL]*len(df.index), mode='lines', name='Upper Limit', line=dict(dash='dash')))
    fig.add_trace(go.Scatter(x=df.index, y=[LL]*len(df.index), mode='lines', name='Lower Limit', line=dict(dash='dash')))

    # Add buy and sell signals
    fig.add_trace(go.Scatter(x=df.index, y=df['zscore'].where(df['asset1_signal'] > 0),
                             mode='markers', name='Buy Signal', marker=dict(color='green', symbol='triangle-up')))

    fig.add_trace(go.Scatter(x=df.index, y=df['zscore'].where(df['asset1_signal'] < 0),
                             mode='markers', name='Sell Signal', marker=dict(color='red', symbol='triangle-down')))

    # Layout options
    fig.update_layout(title=f'Trading Signals for {asset1} and {asset2}',
                      xaxis_title='Date',
                      yaxis_title='Z-Score')

    # Save the figure
    fig.write_html(f'img/signals/{asset1}_{asset2}.html')

    

In [77]:
# Main function
def main(csv_path: str):
    # Read the CSV file
    asset_pairs = pd.read_csv(csv_path)
    
    # Initialize metrics DataFrame
    metrics_df = pd.DataFrame(columns=['Pair Name', 'CAGR', 'Sharpe Ratio', 'Number of Trades', 'Max Drawdown'])

    # Date range for backtesting (Last 6 months)
    end = datetime.now().date()
    start = (datetime.now() - relativedelta(months=6)).date()

    for index, row in asset_pairs.iterrows():
        asset1 = row['Asset1']
        asset2 = row['Asset2']
        beta = row['Beta']
        print("Running Backtest for Asset Pair {asset2} - {beta} * {asset1}".format(asset1=asset1,asset2=asset2,beta=beta))
        print("Downloading Data")
        # Download data
        asset1_data = yf.download(asset1, start=start, end=end, progress=False)['Adj Close']
        asset2_data = yf.download(asset2, start=start, end=end, progress=False)['Adj Close']

        # DataFrame for backtesting
        df_trading = pd.DataFrame({asset1: asset1_data, asset2: asset2_data})
        print("Running Backtest metrics")
        # Backtest and get metrics and signals
        metrics, signals_df = backtest(df_trading, asset1, asset2, beta)

        new_row = pd.DataFrame({
        'Pair Name': [f'{asset1}_{asset2}'],
        'CAGR': [metrics['CAGR']],
        'Sharpe Ratio': [metrics['Sharpe ratio']],
        'Number of Trades': [metrics['Number of Trades']],
        'Max Drawdown': [metrics['Max Drawdown']]
        })

        metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)


        # Save the Plotly graph
        save_plotly_graph(signals_df, asset1, asset2)

    # Save the metrics DataFrame
    metrics_df.to_csv('data/backtest.csv', index=False)

In [78]:
main("data/final_pairs.csv")

Running Backtest for Asset Pair CAH - 0.5084909890953054 * COR
Running Backtest metrics
Running Backtest for Asset Pair BSX - -0.3258377429197281 * CVS
Running Backtest metrics
Running Backtest for Asset Pair CNC - 0.6287906927485161 * CVS
Running Backtest metrics
Running Backtest for Asset Pair BIIB - -1.0250640824360562 * DHR
Running Backtest metrics
Running Backtest for Asset Pair DHR - -0.5908277961593145 * DXCM
Running Backtest metrics
Running Backtest for Asset Pair AMGN - 0.552141142200434 * ELV
Running Backtest metrics
Running Backtest for Asset Pair BSX - 0.1494420459553779 * HCA
Running Backtest metrics
Running Backtest for Asset Pair DHR - -0.2511815428804966 * HCA
Running Backtest metrics
Running Backtest for Asset Pair GSK - 1.7630776257727432 * HLN
Running Backtest metrics
Running Backtest for Asset Pair HCA - 27.933099291546444 * HLN
Running Backtest metrics
Running Backtest for Asset Pair GSK - 0.0893907518723032 * HZNP
Running Backtest metrics
Running Backtest for Asse

In [79]:
backtest_df = pd.read_csv("data/backtest.csv")

In [80]:
backtest_df

,Pair Name,CAGR,Sharpe Ratio,Number of Trades,Max Drawdown
0,COR_CAH,-0.004317,-0.071588,21,-0.033178
1,CVS_BSX,-0.014810,-0.069277,18,-0.060694
2,CVS_CNC,-0.126068,-2.090277,16,-0.084898
3,DHR_BIIB,0.062167,0.932127,10,-0.023677
4,DXCM_DHR,0.033573,0.362039,8,-0.050084
5,ELV_AMGN,-0.074249,-3.100509,11,-0.041641
6,HCA_BSX,-0.077956,-1.191575,15,-0.039769
7,HCA_DHR,-0.084176,-1.685206,17,-0.055241
8,HLN_GSK,-0.018792,-0.131563,10,-0.070173
9,HLN_HCA,0.268340,0.619305,4,-0.256295
